In [1]:
import pandas as pd

In [53]:
class GenRuleMapping:
    def __init__(self):        
        self.start_priority = 1553
        self.mapping_category = "AD"
        self.mapping_provider = "RULE"
        self.ad_channel_names = ["기타", "미디어", "후기/바이럴", "유통채널 내 광고"]
        self.created_by = 3
        self.updated_by = 3

    def get_mapping_name(self, row):
        return f"[임시] {row["brand"]} {row["market"]} {row["item"]}"

    def get_mapping_rule(self, row):
        rule = {"lhs": {"lhs": "ad_channel_name", "rhs": self.ad_channel_names, "_typeTag": "BinFunction", "_functionName": "IN"}, "rhs": {"lhs": {"lhs": "ad_account_name", "rhs": row["brand"], "_typeTag": "BinFunction", "_functionName": "LIKE"}, "rhs": {"lhs": {"lhs": "campaign_name", "rhs": row['market'], "_typeTag": "BinFunction", "_functionName": "LIKE"}, "rhs": {"lhs": "adset_name", "rhs": row["item"], "_typeTag": "BinFunction", "_functionName": "LIKE"}, "_typeTag": "And"}, "_typeTag": "And"}, "_typeTag": "And"}
        rule = str(rule).replace("'", "\"").replace("False", "false")
        
        return rule

    def get_mapping_rule_fe(self, row):
        rule = {"nodeCnt": 4, "isAdvancedCondition": False, "conditionGroupBlocks": [[{"key": 0, "lhs": "ad_channel_name", "rhs": self.ad_channel_names, "_typeTag": "BinFunction", "_functionName": "IN", "groupNodesRelation": "And"}, {"key": 1, "lhs": "ad_account_name", "rhs": row["brand"], "_typeTag": "BinFunction", "_functionName": "LIKE", "groupNodesRelation": "And"}, {"key": 2, "lhs": "campaign_name", "rhs": row["market"], "_typeTag": "BinFunction", "_functionName": "LIKE", "groupNodesRelation": "And"}, {"key": 3, "lhs": "adset_name", "rhs": row["item"], "_typeTag": "BinFunction", "_functionName": "LIKE", "groupNodesRelation": "And"}]], "groupConditionRelations": []}
        rule  = str(rule).replace("'", "\"").replace("False", "false")
        
        return rule

    def get_mapping_sql(self, row):
        return f"(ad_channel_name IN ([''기타'', ''미디어'', ''후기/바이럴'', ''유통채널 내 광고'']) AND (ad_account_name LIKE ''%{row["brand"]}%'' ESCAPE ''\\\\'' AND (campaign_name LIKE ''%{row["market"]}%'' ESCAPE ''\\\\'' AND adset_name LIKE ''%{row["item"]}%'' ESCAPE ''\\\\'')))"
    
    def gen_mapping_row(self, row, priority):

        return f"('{self.mapping_category}', '{self.get_mapping_name(row)}', '{self.mapping_provider}', {priority}, CAST('{self.get_mapping_rule(row)}' AS JSON), CAST('{self.get_mapping_rule_fe(row)}' AS JSON), '{self.get_mapping_sql(row)}'),"
    
    def run(self, df):
        sql_rows = []
        for index, row in df.iterrows():
            priority = self.start_priority + index
            sql_row = self.gen_mapping_row(row, priority)
            sql_rows.append(sql_row)
        return sql_rows
        
        

In [ ]:
df = pd.read_csv("data/source.tsv", sep="\t")

gen_rule_mapping = GenRuleMapping()

result = gen_rule_mapping.run(df)


for sql_row in result:
    print(sql_row)